# Import Libraries

In [1]:
import sys
import os

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
from text_to_sql import (
    TextToSQL,
    Config,
    LLMConfig,
    SLConfig,
    ContextConfig,
    QueryConfig,
)
from dotenv import load_dotenv
from datetime import datetime

import pandas as pd
import os

# Constants

In [3]:
DATABASE = "academic"
MODEL = "gemini-1.5-pro"
PROVIDER = "gemini"
# TOTAL_TABLES = 14
# TOTAL_TABLES = 15
# TOTAL_TABLES = 18
TOTAL_TABLES = 22

# Load Environment

In [4]:
load_dotenv()

True

# Set Timestamp Experiment

In [5]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
output_dir = f"../files/experiment_result/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

# Config

In [6]:
db_key = DATABASE.upper().replace("-", "_")
provider_key = PROVIDER.upper().replace("-", "_")

config = Config(
    max_retry_attempt=5,
    rewriter_config=LLMConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
    ),
    query_generator_config=LLMConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
    ),
    schema_linker_config=SLConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
        schema_path=f"../files/schema/{DATABASE}.txt",
        metadata_path=f"../files/metadata/{DATABASE}.json",
    ),
    retrieve_context_config=ContextConfig(data_path=f"../files/dataset/dataset_{DATABASE}.csv"),
    query_executor_config = QueryConfig(
        host=os.getenv(f"DB_HOST_{db_key}"),
        database=os.getenv(f"DB_DATABASE_{db_key}"),
        user=os.getenv(f"DB_USER_{db_key}"),
        password=os.getenv(f"DB_PASSWORD_{db_key}"),
        port=os.getenv(f"DB_PORT_{db_key}"),
    ),
)

# Model

In [7]:
text_to_sql_model = TextToSQL(config=config)

Initializing API client for gemini using model gemini-1.5-pro.
Initializing API client for gemini using model gemini-1.5-pro.
Initializing API client for gemini using model gemini-1.5-pro.


c:\Users\michj\AppData\Local\Programs\Python\Python39\lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


# Import Dataset

In [8]:
dataset = pd.read_csv(f"../files/dataset/dataset_schema_linker_{DATABASE}.csv")

In [9]:
dataset["tables_used"] = dataset["tables_used"].apply(eval)

In [10]:
dataset

,prompt,tables_used
0,Can you tell me which faculty did the most res...,"[fakultas, proyekpenelitian]"
1,I’m curious which lecturer guides the most stu...,"[mahasiswa, dosen]"
2,Which student joined the most sessions? I want...,"[mahasiswa, catatankehadiran]"
3,What are the top 5 most popular courses? Show ...,"[matakuliah, sesi, enrolmenkelas]"
4,Which faculty uses the most facilities? I need...,"[fakultas, fasilitasfakultas]"
...,...,...
95,Mahasiswa mana yang dosennya juga nulis publik...,"[mahasiswa, penulispublikasi, dosen]"
96,Dosen mana aja sih yang aktif ngajar dan jadi ...,"[sesi, mahasiswa, proyekpenelitian, dosen]"
97,Fasilitas mana yang dipake semua fakultas? Kol...,"[fakultas, fasilitas, fasilitasfakultas]"
98,Siapa aja mahasiswa yang ikut proyek penelitia...,"[partisipasipenelitian, mahasiswa, proyekpenel..."


# Experiment

In [11]:
def extract_table_names_from_schema(schema: dict) -> list:
    """
    Given a schema dictionary (from predict_schema_only),
    return a list of lowercase table names.
    """
    return schema.get("potential_related_tables", [])

In [12]:
def calculate_reduction(predicted_tables):
    return 1 - (len(predicted_tables) / TOTAL_TABLES)

In [13]:
accuracies = []
predicted_tables_list = []

for _, row in dataset.iterrows():
    prompt = row["prompt"]
    true_tables = set(row["tables_used"])

    predicted_schema = text_to_sql_model.predict_schema_only(prompt)
    predicted_tables = set(extract_table_names_from_schema(predicted_schema))

    # Save predicted tables
    predicted_tables_list.append(list(predicted_tables))

    # Calculate accuracy
    intersection_count = len(true_tables.intersection(predicted_tables))
    total_true_tables = len(true_tables)

    if total_true_tables > 0:
        accuracy = intersection_count / total_true_tables
    else:
        accuracy = 1.0 if not predicted_tables else 0.0

    accuracies.append(accuracy)

In [14]:
dataset["predicted_tables"] = predicted_tables_list
dataset["schema_accuracy"] = accuracies
dataset['schema_reduction'] = dataset['predicted_tables'].apply(calculate_reduction)

In [15]:
final_accuracy = sum(accuracies) / len(accuracies)
print(f"Schema Prediction Accuracy (Intersection-Based): {final_accuracy:.2%}")

average_reduction = dataset['schema_reduction'].mean()
print(f"Average Schema Reduction: {average_reduction:.2%}")

Schema Prediction Accuracy (Intersection-Based): 87.02%
Average Schema Reduction: 51.86%


# Save

In [16]:
dataset.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_schema_linker.csv", index=False)

In [17]:
dataset

,prompt,tables_used,predicted_tables,schema_accuracy,schema_reduction
0,Can you tell me which faculty did the most res...,"[fakultas, proyekpenelitian]","[jurusan, mahasiswa, proyekpenelitian, partisi...",1.000000,0.681818
1,I’m curious which lecturer guides the most stu...,"[mahasiswa, dosen]","[jurusan, mahasiswa, organisasiklub, keanggota...",1.000000,0.545455
2,Which student joined the most sessions? I want...,"[mahasiswa, catatankehadiran]","[jurusan, mahasiswa, organisasiklub, keanggota...",1.000000,0.590909
3,What are the top 5 most popular courses? Show ...,"[matakuliah, sesi, enrolmenkelas]","[kelas, jurusan, mahasiswa, organisasiklub, ke...",0.333333,0.636364
4,Which faculty uses the most facilities? I need...,"[fakultas, fasilitasfakultas]","[jurusan, mahasiswa, organisasiklub, fasilitas...",1.000000,0.545455
...,...,...,...,...,...
95,Mahasiswa mana yang dosennya juga nulis publik...,"[mahasiswa, penulispublikasi, dosen]","[ujian, jurusan, mahasiswa, organisasiklub, ke...",1.000000,0.318182
96,Dosen mana aja sih yang aktif ngajar dan jadi ...,"[sesi, mahasiswa, proyekpenelitian, dosen]","[jurusan, mahasiswa, organisasiklub, keanggota...",1.000000,0.500000
97,Fasilitas mana yang dipake semua fakultas? Kol...,"[fakultas, fasilitas, fasilitasfakultas]","[jurusan, mahasiswa, organisasiklub, fasilitas...",1.000000,0.636364
98,Siapa aja mahasiswa yang ikut proyek penelitia...,"[partisipasipenelitian, mahasiswa, proyekpenel...","[jurusan, mahasiswa, organisasiklub, keanggota...",1.000000,0.500000


In [18]:
dataset_1 = dataset[dataset["schema_accuracy"] == 1]

In [19]:
dataset_1

,prompt,tables_used,predicted_tables,schema_accuracy,schema_reduction
0,Can you tell me which faculty did the most res...,"[fakultas, proyekpenelitian]","[jurusan, mahasiswa, proyekpenelitian, partisi...",1.0,0.681818
1,I’m curious which lecturer guides the most stu...,"[mahasiswa, dosen]","[jurusan, mahasiswa, organisasiklub, keanggota...",1.0,0.545455
2,Which student joined the most sessions? I want...,"[mahasiswa, catatankehadiran]","[jurusan, mahasiswa, organisasiklub, keanggota...",1.0,0.590909
4,Which faculty uses the most facilities? I need...,"[fakultas, fasilitasfakultas]","[jurusan, mahasiswa, organisasiklub, fasilitas...",1.0,0.545455
5,Which course has the best average exam score? ...,"[matakuliah, hasilujian, ujian]","[jurusan, mahasiswa, bukuperpustakaan, peminja...",1.0,0.500000
...,...,...,...,...,...
95,Mahasiswa mana yang dosennya juga nulis publik...,"[mahasiswa, penulispublikasi, dosen]","[ujian, jurusan, mahasiswa, organisasiklub, ke...",1.0,0.318182
96,Dosen mana aja sih yang aktif ngajar dan jadi ...,"[sesi, mahasiswa, proyekpenelitian, dosen]","[jurusan, mahasiswa, organisasiklub, keanggota...",1.0,0.500000
97,Fasilitas mana yang dipake semua fakultas? Kol...,"[fakultas, fasilitas, fasilitasfakultas]","[jurusan, mahasiswa, organisasiklub, fasilitas...",1.0,0.636364
98,Siapa aja mahasiswa yang ikut proyek penelitia...,"[partisipasipenelitian, mahasiswa, proyekpenel...","[jurusan, mahasiswa, organisasiklub, keanggota...",1.0,0.500000


In [20]:
new_dataset = pd.read_csv(f"../files/dataset/dataset_{DATABASE}.csv")

In [21]:
new_dataset

,Question,Answer,Summary,Alternative Prompt 1 (English),Alternative Prompt 2 (Bahasa Indonesia),Expected Result
0,Which faculty has the most research projects? ...,"SELECT f.nama, COUNT(p.id_proyek) AS total_pro...",This SQL query identifies the faculty with the...,Can you tell me which faculty did the most res...,Fakultas mana yang paling banyak bikin proyek ...,"['nama', 'total_proyek']"
1,Which lecturer supervises the most students? I...,"SELECT d.nama, COUNT(m.id_mahasiswa) AS total_...",This SQL query identifies the lecturer supervi...,I’m curious which lecturer guides the most stu...,Dosen mana yang ngebimbing mahasiswa paling ba...,"['nama', 'total_mahasiswa_dibimbing']"
2,Which student has attended the most sessions? ...,"SELECT m.nama, COUNT(k.id_kehadiran) AS total_...",This SQL query identifies the student with the...,Which student joined the most sessions? I want...,Mahasiswa mana yang paling rajin ikut sesi kel...,"['nama', 'total_kehadiran']"
3,What are the top 5 courses with the highest nu...,"SELECT mk.judul, COUNT(e.id_mahasiswa) AS tota...",This SQL query identifies the top 5 most popul...,What are the top 5 most popular courses? Show ...,Top 5 matkul yang paling banyak diambil siapa ...,"['judul', 'total_mahasiswa']"
4,Which faculty uses the most facilities? If the...,"SELECT f.nama, COUNT(ff.id_fasilitas) AS total...",This SQL query identifies the faculty with the...,Which faculty uses the most facilities? I need...,Fakultas mana yang paling banyak pake fasilita...,"['nama', 'total_fasilitas']"
5,Which course has the highest average exam scor...,"SELECT mk.judul, AVG(h.nilai_diperoleh) AS rat...",This SQL query identifies the course with the ...,Which course has the best average exam score? ...,Matkul mana yang nilai ujiannya paling tinggi ...,"['judul', 'rata_rata_nilai']"
6,Which lecturer has authored the most publicati...,"SELECT d.nama, COUNT(p.id_publikasi) AS total_...",This SQL query identifies the most published f...,Who is the lecturer with the most publications...,Dosen mana yang paling banyak nulis publikasi?...,"['nama', 'total_publikasi']"
7,Which student has participated in the most res...,"SELECT m.nama, COUNT(pp.id_proyek) AS total_pa...",This SQL query identifies the student with the...,Which student has joined the most research pro...,Mahasiswa mana yang paling sering ikut proyek ...,"['nama', 'total_partisipasi']"
8,Which student has borrowed the most books? If ...,"SELECT id_peminjam, COUNT(*) AS total_peminjam...",This SQL query identifies the most active libr...,Who’s borrowed the most books among students? ...,Mahasiswa mana yang paling sering minjem buku?...,"['id_peminjam', 'total_peminjaman']"
9,Which club has the most active members? If the...,"SELECT k.nama, COUNT(kk.id_mahasiswa) AS total...",This SQL query identifies the most popular stu...,Which club has the most members? Show the name...,Klub mana yang anggotanya paling banyak? Kasih...,"['nama', 'total_anggota']"


In [22]:
prompt_set = set(dataset_1['prompt'])

def match_prompt(row):
    alt1 = row.get("Alternative Prompt 1 (English)", "")
    alt2 = row.get("Alternative Prompt 2 (Bahasa Indonesia)", "")
    return alt1 in prompt_set and alt2 in prompt_set

filtered_df = new_dataset[new_dataset.apply(match_prompt, axis=1)]
filtered_df

,Question,Answer,Summary,Alternative Prompt 1 (English),Alternative Prompt 2 (Bahasa Indonesia),Expected Result
0,Which faculty has the most research projects? ...,"SELECT f.nama, COUNT(p.id_proyek) AS total_pro...",This SQL query identifies the faculty with the...,Can you tell me which faculty did the most res...,Fakultas mana yang paling banyak bikin proyek ...,"['nama', 'total_proyek']"
1,Which lecturer supervises the most students? I...,"SELECT d.nama, COUNT(m.id_mahasiswa) AS total_...",This SQL query identifies the lecturer supervi...,I’m curious which lecturer guides the most stu...,Dosen mana yang ngebimbing mahasiswa paling ba...,"['nama', 'total_mahasiswa_dibimbing']"
2,Which student has attended the most sessions? ...,"SELECT m.nama, COUNT(k.id_kehadiran) AS total_...",This SQL query identifies the student with the...,Which student joined the most sessions? I want...,Mahasiswa mana yang paling rajin ikut sesi kel...,"['nama', 'total_kehadiran']"
4,Which faculty uses the most facilities? If the...,"SELECT f.nama, COUNT(ff.id_fasilitas) AS total...",This SQL query identifies the faculty with the...,Which faculty uses the most facilities? I need...,Fakultas mana yang paling banyak pake fasilita...,"['nama', 'total_fasilitas']"
6,Which lecturer has authored the most publicati...,"SELECT d.nama, COUNT(p.id_publikasi) AS total_...",This SQL query identifies the most published f...,Who is the lecturer with the most publications...,Dosen mana yang paling banyak nulis publikasi?...,"['nama', 'total_publikasi']"
7,Which student has participated in the most res...,"SELECT m.nama, COUNT(pp.id_proyek) AS total_pa...",This SQL query identifies the student with the...,Which student has joined the most research pro...,Mahasiswa mana yang paling sering ikut proyek ...,"['nama', 'total_partisipasi']"
9,Which club has the most active members? If the...,"SELECT k.nama, COUNT(kk.id_mahasiswa) AS total...",This SQL query identifies the most popular stu...,Which club has the most members? Show the name...,Klub mana yang anggotanya paling banyak? Kasih...,"['nama', 'total_anggota']"
11,List all faculties along with their total numb...,"SELECT f.nama,\n COUNT(DISTINCT p.id_proyek) ...",This SQL query provides a comprehensive resear...,Can you list all faculties and show how many r...,Tolong tampilin semua fakultas beserta total p...,"['nama', 'total_proyek', 'total_publikasi', 't..."
14,Which students have participated in more than ...,"SELECT m.id_mahasiswa, m.nama,\n COUNT(DISTIN...",This SQL query identifies highly engaged stude...,Which students joined more than 2 projects and...,Siapa aja mahasiswa yang ikut lebih dari 2 pro...,"['id_mahasiswa', 'nama', 'total_proyek', 'tota..."
16,Find the lecturers who have both supervised st...,"SELECT d.id_dosen, d.nama,\n COUNT(DISTINCT m...",This SQL query evaluates faculty performance b...,Find me lecturers who both supervise students ...,Dosen mana yang bimbing mahasiswa sekaligus bi...,"['id_dosen', 'nama', 'total_mahasiswa_bimbinga..."


In [23]:
len(filtered_df)

22

In [24]:
MAX_RETRIES = 5
RETRY_DELAY = 2

In [25]:
import time
import ast

EA = 0
total_questions = len(filtered_df) * 2
results_list = []

for idx, row in filtered_df.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.generate_baseline(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: Can you tell me which faculty did the most research projects? I just need their name (nama) and the number of projects (total_proyek).
Generated SQL Query: SELECT F.nama, COUNT(P.id_proyek) AS total_proyek
FROM Fakultas AS F
JOIN ProyekPenelitian AS P ON F.id_fakultas = P.id_fakultas
GROUP BY F.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 1.2: Fakultas mana yang paling banyak bikin proyek penelitian? Tampilkan nama (nama) dan jumlah proyeknya (total_proyek).
Generated SQL Query: SELECT F.nama, COUNT(P.id_proyek) AS total_proyek FROM Fakultas AS F JOIN ProyekPenelitian AS P ON F.id_fakultas = P.id_fakultas GROUP BY F.nama ORDER BY total_proyek DESC LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 2.1: I’m curious which lecturer guides the most students. Please show their name (nama) and how many students they supervise (total_mahasiswa_dibimbing).
Generated SQL Query: SELECT d.nama, COUNT(m.id_pembimbing) AS t

In [26]:
df_results_partial_baseline = pd.DataFrame(results_list)
df_results_partial_baseline.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_partial_baseline.csv", index=False)

In [27]:
import time
import ast

EA = 0
total_questions = len(filtered_df) * 2
results_list = []

for idx, row in filtered_df.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.predict_sql_schema_only(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: Can you tell me which faculty did the most research projects? I just need their name (nama) and the number of projects (total_proyek).


Generated SQL Query: SELECT F.nama, COUNT(PP.id_proyek) AS total_proyek
FROM Fakultas AS F
JOIN ProyekPenelitian AS PP ON F.id_fakultas = PP.id_fakultas
GROUP BY F.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 1.2: Fakultas mana yang paling banyak bikin proyek penelitian? Tampilkan nama (nama) dan jumlah proyeknya (total_proyek).
Generated SQL Query: SELECT f.nama, COUNT(p.id_proyek) AS total_proyek
FROM Fakultas f
JOIN ProyekPenelitian p ON f.id_fakultas = p.id_fakultas
GROUP BY f.nama
ORDER BY total_proyek DESC
LIMIT 1;
Execution Accuracy: 1.0000

Processing Question 2.1: I’m curious which lecturer guides the most students. Please show their name (nama) and how many students they supervise (total_mahasiswa_dibimbing).
Generated SQL Query: SELECT d.nama, COUNT(m.id_pembimbing) AS total_mahasiswa_dibimbing
FROM Dosen AS d
JOIN Mahasiswa AS m ON d.id_dosen = m.id_pembimbing
GROUP BY d.nama
ORDER BY total_mahasiswa_dibimbing DESC
LIMIT 1;
Execut

In [28]:
df_results_partial_schema_linker = pd.DataFrame(results_list)
df_results_partial_schema_linker.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_partial_schema_linker.csv", index=False)